In [1]:
import time
import numpy as np
import torch
import torchaudio

import matplotlib.pyplot as plt
import seaborn as sns

from itertools import groupby
from operator import itemgetter
from matplotlib.collections import LineCollection

device = 'cuda'

In [2]:
from speech_parser import Audio
from speech_parser import SileroVAD
from speech_parser import OnlineSpeakerClustering
from speech_parser import MSDD

/home/raid/miniconda3/envs/nemo/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
from utils import load_audio

In [4]:
msdd = MSDD(
    threshold=0.8
)
titanet_l = msdd.speech_embedding_model
vad = SileroVAD(
    threshold=0.5
)
osc = OnlineSpeakerClustering()

scales = [1.5, 1.25, 1.0, 0.75, 0.5]
hops = [scale/4 for scale in scales]
a = Audio(
    scales, 
    hops, 
    speech_embedding_model=titanet_l,
    voice_activity_detection_model=vad,
    multi_scale_diarization_model=msdd,
    speaker_clustering=osc
)

[NeMo I 2025-02-21 14:19:46 cloud:58] Found existing object /home/raid/.cache/torch/NeMo/NeMo_2.1.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.
[NeMo I 2025-02-21 14:19:46 cloud:64] Re-using file from: /home/raid/.cache/torch/NeMo/NeMo_2.1.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo
[NeMo I 2025-02-21 14:19:46 common:826] Instantiating model from pre-trained checkpoint


[NeMo W 2025-02-21 14:19:48 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: true
    
[NeMo W 2025-02-21 14:19:48 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: false
    
[NeMo W 2025-02-21 14:19:48 modelPT:189] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple

[NeMo I 2025-02-21 14:19:48 features:305] PADDING: 16
[NeMo I 2025-02-21 14:19:48 features:305] PADDING: 16
[NeMo I 2025-02-21 14:19:49 save_restore_connector:275] Model EncDecDiarLabelModel was successfully restored from /home/raid/.cache/torch/NeMo/NeMo_2.1.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.


In [5]:
# waveform, sr = load_audio('test.wav')
waveform, sr = load_audio('toefl_eg.mp3')
waveform = waveform[0]

In [6]:
proba, labels = a(waveform[:500_000])

In [7]:
proba

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.,

In [8]:
proba, labels = a(waveform[500_000:1_000_000])

In [9]:
proba, labels = a(waveform[1_000_000:4_000_000])

In [10]:
# a(waveform[4_000_000:5_000_000])

In [30]:
merged = a.get_merged_speaker_segments()

new seg
tensor(0.9096, device='cuda:0')
17.625
new seg
tensor(0.8424, device='cuda:0')
12.875
new seg
tensor(0.9019, device='cuda:0')
19.125
new seg
tensor(0.8902, device='cuda:0')
11.0
new seg
tensor(0.6928, device='cuda:0')
0.625
new seg
tensor(0.2580, device='cuda:0')
0.5
new seg
tensor(0.6704, device='cuda:0')
0.625
new seg
tensor(0.2560, device='cuda:0')
0.625
new seg
tensor(0.8220, device='cuda:0')
0.5
new seg
tensor(1., device='cuda:0')
0.625
new seg
tensor(0.9177, device='cuda:0')
58.375
new seg
tensor(0.1373, device='cuda:0')
0.75
new seg
tensor(0.0660, device='cuda:0')
0.5
new seg
tensor(1., device='cuda:0')
0.5
new seg
tensor(0.8224, device='cuda:0')
14.125
new seg
tensor(0.7845, device='cuda:0')
2.125
new seg
tensor(0.7882, device='cuda:0')
4.75
new seg
tensor(0.7888, device='cuda:0')
4.625
new seg
tensor(0.7618, device='cuda:0')
10.25
new seg
tensor(0.3724, device='cuda:0')
1.375
new seg
tensor(0.7200, device='cuda:0')
2.75
new seg
tensor(0.4208, device='cuda:0')
0.625
new

In [27]:
a.max_silence_per_segment_pct = 0.5

In [33]:
[s.duration for s in merged]

[17.625,
 12.875,
 19.125,
 11.0,
 58.375,
 14.125,
 2.125,
 4.75,
 4.625,
 10.25,
 2.75,
 10.625,
 1.0,
 1.125,
 10.25,
 2.125,
 5.625,
 4.125,
 6.0,
 10.625,
 9.0,
 5.25,
 9.375,
 6.375,
 8.625,
 5.875]

In [40]:
merged

[SpeakerSegment(start=0, end=17.625, data=tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.0015, -0.0009, -0.0007],
        device='cuda:0'), mask=tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0'), speaker=0, transcription=None),
 SpeakerSegment(start=17.875, end=30.75, data=tensor([-0.0037, -0.0012,  0.0005,  ...,  0.0074,  0.0059,  0.0018],
        device='cuda:0'), mask=tensor([0., 0., 0.,  ..., 1., 1., 1.], device='cuda:0'), speaker=0, transcription=None),
 SpeakerSegment(start=31.25, end=50.375, data=tensor([0.0137, 0.0135, 0.0104,  ..., 0.0037, 0.0032, 0.0023], device='cuda:0'), mask=tensor([1., 1., 1.,  ..., 0., 0., 0.], device='cuda:0'), speaker=0, transcription=None),
 SpeakerSegment(start=50.875, end=61.875, data=tensor([ 1.6737e-03,  3.6514e-04, -5.9805e-05,  ..., -1.1843e-03,
         -1.0885e-03, -7.7725e-04], device='cuda:0'), mask=tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0'), speaker=0, transcription=None),
 SpeakerSegment(start=62.5, end=120.875, data=tensor

In [38]:
i = 13
torchaudio.save('segmented.wav', merged[i].data.unsqueeze(0).cpu(), 16_000)
merged[i].speaker

2

In [39]:
merged[i].duration

1.125

In [56]:
a.base_scale_segments[1].mask.sum()

tensor(7168., device='cuda:0')

In [37]:
from speech_parser.audio import SpeakerSegment

In [59]:
sr = 16_000

def merge_segments(segments, speaker, sampling_rate=16000):
    
    first_segment = segments[0]
    start = first_segment.start
    end = first_segment.start
    normal_segment_duration = first_segment.end - first_segment.start
    waveforms = []
    
    for segment in segments:
        increment = segment.end - end
        assert increment <= normal_segment_duration, f"Issue in the matching section of get_merged_speaker_segments(), {increment} > {normal_segment_duration}"

        concat_length = int(increment*sampling_rate)
        waveforms.append(segment.data[-concat_length:])
        
        end = segment.end
        
    waveform = torch.cat(waveforms)
    return SpeakerSegment(start, end, waveform, speaker)
    # print([segment.start for segment in segments])
    # pass

In [60]:
segment.data.shape

torch.Size([8000])

In [61]:
active_speakers  = {}

merged_segments = []

for segment in a.base_scale_segments:
    curr_active_speakers = list(active_speakers.items())
    for speaker, active_segments in curr_active_speakers:
        # end the speaker segment if the start of the new segment is past the end of the old
        if (speaker in segment.speakers) and (segment.start <= active_segments[-1].end):
            active_speakers[speaker].append(segment)
        else:
            # merge and end speakers
            merged_segments.append(merge_segments(active_segments, speaker))
            active_speakers.pop(speaker)
        
    for speaker in segment.speakers:
        if speaker not in active_speakers:
            # start speaker
            active_speakers[speaker] = [segment]

In [62]:
merged_segments

[SpeakerSegment(start=62.5, end=120.75, data=tensor([-0.0164, -0.0267,  0.0036,  ..., -0.0011, -0.0012, -0.0008],
        device='cuda:0'), speaker=0, transcription=None),
 SpeakerSegment(start=120.75, end=134.75, data=tensor([-0.0006, -0.0006, -0.0004,  ...,  0.0000,  0.0000,  0.0000],
        device='cuda:0'), speaker=0, transcription=None),
 SpeakerSegment(start=136.25, end=138.5, data=tensor([ 1.0500e-04,  3.5520e-05, -9.4904e-07,  ..., -1.4732e-02,
         -1.2964e-02, -1.2140e-02], device='cuda:0'), speaker=0, transcription=None),
 SpeakerSegment(start=136.25, end=141.0, data=tensor([ 1.0500e-04,  3.5520e-05, -9.4904e-07,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00], device='cuda:0'), speaker=1, transcription=None),
 SpeakerSegment(start=141.25, end=145.75, data=tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -7.6305e-05,
         -5.3419e-05, -3.3926e-05], device='cuda:0'), speaker=1, transcription=None),
 SpeakerSegment(start=146.0, end=156.0, data=tensor([0., 0.

In [66]:
merged_segments[-2].data

tensor(0.4973, device='cuda:0')

In [30]:
active_speakers

{}

In [19]:
def process_timeline(data):
    # Convert None to empty list for consistency
    timeline = [[] if x is None else sorted(x) for x in data]
    return timeline

def merge_segments(timeline):
    merged_segments = []
    
    # Find continuous segments for each speaker
    for speaker in set([spk for t in timeline for spk in t]):
        start_idx = None
        
        for t, speakers in enumerate(timeline):
            if speaker in speakers:
                if start_idx is None:
                    start_idx = t
            elif start_idx is not None:
                # Add segment
                merged_segments.append({
                    'speaker': speaker,
                    'start': start_idx * 0.25,  # Convert to seconds
                    'end': t * 0.25,            # Convert to seconds
                    'duration': (t - start_idx) * 0.25
                })
                start_idx = None
        
        # Handle segment that goes until the end
        if start_idx is not None:
            merged_segments.append({
                'speaker': speaker,
                'start': start_idx * 0.25,
                'end': len(timeline) * 0.25,
                'duration': (len(timeline) - start_idx) * 0.25
            })
    
    return sorted(merged_segments, key=lambda x: (x['start'], x['speaker']))

def generate_rttm(segments):
    rttm_lines = []
    
    for seg in segments:
        rttm_line = f"SPEAKER unknown 1 {seg['start']:.3f} {seg['duration']:.3f} <NA> <NA> SPEAKER_{seg['speaker']} <NA> <NA>"
        rttm_lines.append(rttm_line)
    
    return "\n".join(rttm_lines)

def create_visualization(segments, total_duration):
    plt.figure(figsize=(20, 5))
    
    # Get unique speakers and assign them y-coordinates
    unique_speakers = sorted(set(seg['speaker'] for seg in segments))
    speaker_to_y = {speaker: i for i, speaker in enumerate(unique_speakers)}
    
    # Create line segments for each speaker
    for speaker in unique_speakers:
        speaker_segments = [seg for seg in segments if seg['speaker'] == speaker]
        
        for seg in speaker_segments:
            plt.hlines(
                y=speaker_to_y[seg['speaker']],
                xmin=seg['start'],
                xmax=seg['end'],
                linewidth=4,
                label=f"Speaker {speaker}"
            )
    
    # Set axis limits to show the full timeline
    plt.xlim(0, total_duration)
    plt.ylim(-0.5, len(unique_speakers) - 0.5)
    
    # Set y-axis labels
    plt.yticks(
        range(len(unique_speakers)),
        [f"Speaker {speaker}" for speaker in unique_speakers]
    )
    
    # Remove duplicate labels in legend
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc='center left', bbox_to_anchor=(1, 0.5))
    
    # Customize the plot
    plt.xlabel("Time (seconds)")
    plt.ylabel("Speakers")
    plt.title("Speaker Diarization Timeline")
    plt.grid(True, alpha=0.3)
    
    # Add some padding to the plot
    plt.margins(x=0.02)
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    
    # Save the plot
    plt.savefig('speaker_diarization.png', bbox_inches='tight')
    plt.close()

# Use your existing code to process the data and generate segments
timeline = process_timeline(timeline)
merged_segments = merge_segments(timeline)

# Calculate total duration in seconds
total_duration = len(timeline) * 0.25

# Generate RTTM
rttm_content = generate_rttm(merged_segments)
with open('output.rttm', 'w') as f:
    f.write(rttm_content)

# Create visualization with total duration
create_visualization(merged_segments, total_duration)

# Print segments for verification
for seg in merged_segments:
    print(f"Speaker {seg['speaker']}: {seg['start']:.2f}s - {seg['end']:.2f}s (duration: {seg['duration']:.2f}s)")

Speaker 0: 62.00s - 120.00s (duration: 58.00s)
Speaker 0: 120.25s - 134.00s (duration: 13.75s)
Speaker 0: 135.75s - 137.75s (duration: 2.00s)
Speaker 1: 135.75s - 140.25s (duration: 4.50s)
Speaker 1: 140.75s - 145.00s (duration: 4.25s)
Speaker 2: 145.50s - 155.25s (duration: 9.75s)
Speaker 2: 155.50s - 156.50s (duration: 1.00s)
Speaker 1: 156.75s - 160.00s (duration: 3.25s)
Speaker 2: 159.00s - 168.75s (duration: 9.75s)
Speaker 1: 168.50s - 178.25s (duration: 9.75s)
Speaker 2: 171.75s - 172.75s (duration: 1.00s)
Speaker 2: 175.25s - 175.75s (duration: 0.50s)
Speaker 2: 178.00s - 181.75s (duration: 3.75s)
Speaker 2: 182.00s - 187.25s (duration: 5.25s)
Speaker 1: 187.50s - 191.25s (duration: 3.75s)
Speaker 2: 191.00s - 191.25s (duration: 0.25s)
Speaker 1: 191.50s - 191.75s (duration: 0.25s)
Speaker 2: 191.50s - 197.00s (duration: 5.50s)
Speaker 1: 196.50s - 206.75s (duration: 10.25s)
Speaker 1: 207.00s - 215.75s (duration: 8.75s)
Speaker 2: 215.25s - 220.25s (duration: 5.00s)
Speaker 1: 